In [1]:
!pip install tensorflow tensorflow_datasets

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Set a shorter path for TensorFlow Datasets
import os
os.environ['TFDS_DATA_DIR'] = 'tensorflow_datasets'

# Load the Plant Village dataset
(ds_train, ds_test), ds_info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

#load from existing
#ds_train = tfds.load('plant_village', split='train[:80%]', as_supervised=True)
#ds_test = tfds.load('plant_village', split='train[80%:]', as_supervised=True)


In [ ]:
# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (128, 128))
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    return image, label

In [ ]:
# Apply preprocessing
ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Build a simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(ds_info.features['label'].num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(ds_train, epochs=10, validation_data=ds_test)

In [ ]:
# Evaluate the model
loss, acc = model.evaluate(ds_test)
print(f"Test Loss: {loss:.2f}")
print(f"Test Accuracy: {acc:.2f}")


In [ ]:
import matplotlib.pyplot as plt

# Display 5 sample images from the training set
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(ds_train.take(5)):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow((image[0].numpy() * 255).astype("uint8"))  # Multiply by 255 for conversion
    plt.title(ds_info.features['label'].int2str(label[0].numpy()))
    plt.axis("off")
plt.show()



In [ ]:

# Predict for one test image using its index
index = 69  # Change this to the desired index
image, label = list(ds_test.take(index + 1))[-1]  # Get the (index+1)th element

# Display the test image
plt.imshow((image[0].numpy() * 255).astype("uint8"))  # Multiply by 255 for conversion
plt.title(ds_info.features['label'].int2str(label[0].numpy()))
plt.axis("off")
plt.show()

# Make the prediction
prediction = model.predict(tf.expand_dims(image[0], axis=0))
predicted_label = tf.argmax(prediction, axis=1).numpy()[0]

# Print the prediction
print("Predicted Label:", ds_info.features['label'].int2str(predicted_label))